### Trip Volume Analysis
## How has the volume of Bixi trips changed over time?

using open data from bixi's website from 2014 to 2024

In [3]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as ss

## Part 1 : Cleaning data

In [4]:
data_folder = 'raw_data/'
files = os.listdir(data_folder)
csv_files = [f for f in files]
csv_files

['.DS_Store',
 'BixiMontrealRentals2016',
 'BixiMontrealRentals2018',
 'BixiMontrealRentals2020',
 'BixiMontrealRentals2021',
 'BixiMontrealRentals2019',
 'BixiMontrealRentals2017',
 'BixiMontrealRentals2015',
 'BixiMontrealRentals2023',
 'BixiMontrealRentals2024',
 'BixiMontrealRentals2022',
 'BixiMontrealRentals2014']

### Cleaning 2014 - 2019

In [5]:
def cleanMonthlyDataFor(year):
    # Path to the folder containing year data files
    data_folder = f'raw_data/BixiMontrealRentals{year}/'

    # List of files to be processed
    files = [f for f in os.listdir(data_folder) if f.startswith(f'OD_{year}') and f.endswith('.csv')]

    # List to store month and row count (volume)
    data = []

    # Loop through each file, read it, and count the rows
    for file in files:
        # Extract the month from the filename (e.g., 'OD_{year}-04.csv' -> 04)
        month = file.split('-')[1].split('.')[0]
        
        # Read the file into a DataFrame
        file_path = os.path.join(data_folder, file)
        df = pd.read_csv(file_path)
        
        # Count the number of rows (excluding the header)
        volume = len(df)
        
        # Store the month and volume in the data list
        data.append({'month': int(month), 'volume': volume})

    # Create a DataFrame from the collected data
    result_df = pd.DataFrame(data)

    # Sort the DataFrame by month to ensure the order is correct
    result_df = result_df.sort_values(by='month')

    # Define the output path for the cleaned file
    output_path = f'question1_data/{year}_cleaned.csv'

    # Save the cleaned data to a new CSV file
    result_df.to_csv(output_path, index=False)

    print(f"Cleaned data written to {output_path}")


In [6]:
for year in range(2014,2020):
    cleanMonthlyDataFor(year)

Cleaned data written to question1_data/2014_cleaned.csv
Cleaned data written to question1_data/2015_cleaned.csv
Cleaned data written to question1_data/2016_cleaned.csv
Cleaned data written to question1_data/2017_cleaned.csv
Cleaned data written to question1_data/2018_cleaned.csv


/var/folders/3s/z5bjp2_j3_525q4w1h0qhgpw0000gn/T/ipykernel_4023/1152578176.py:18: DtypeWarning: Columns (1,3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


Cleaned data written to question1_data/2019_cleaned.csv


### Cleaning 2020 - 2021

In [7]:
def cleanYearlyDataFor(year):
    # Path to the {year} data file
    data_file = f'raw_data/BixiMontrealRentals{year}/OD_{year}.csv'

    # Read the CSV file into a DataFrame
    df = pd.read_csv(data_file)

    # Convert the 'start_date' column to datetime format
    df['start_date'] = pd.to_datetime(df['start_date'])

    # Extract the month from the 'start_date' column
    df['month'] = df['start_date'].dt.month

    # Group by month and count the number of rows (volume) for each month
    monthly_volume = df.groupby('month').size().reset_index(name='volume')

    # Define the output path for the cleaned file
    output_path = f'question1_data/{year}_cleaned.csv'

    # Save the cleaned data to a new CSV file
    monthly_volume.to_csv(output_path, index=False)

    print(f"Cleaned data written to {output_path}")



In [8]:
for year in range(2020,2022):
    cleanYearlyDataFor(year)

Cleaned data written to question1_data/2020_cleaned.csv
Cleaned data written to question1_data/2021_cleaned.csv


### Cleaning 2022 - 2024

In [11]:
def cleanYearlyLinuxTStmpDataFor(year):
    # Path to the {year} data file
    data_file = f'raw_data/BixiMontrealRentals{year}/OD_{year}.csv'

    # Read the CSV file into a DataFrame
    df = pd.read_csv(data_file)
        
    # Convert 'STARTTIMEMS' (Linux timestamp in milliseconds) to datetime
    df['start_time'] = pd.to_datetime(df['STARTTIMEMS'], unit='ms')

    # Extract the month from the 'start_time' column
    df['month'] = df['start_time'].dt.month

    # Group by month and count the number of rows (volume) for each month
    monthly_volume = df.groupby('month').size().reset_index(name='volume')

    # Define the output path for the cleaned file
    output_path = f'question1_data/{year}_cleaned.csv'

    # Save the cleaned data to a new CSV file
    monthly_volume.to_csv(output_path, index=False)

    print(f"Cleaned data written to {output_path}")


In [13]:
for year in range(2022,2025):
    cleanYearlyLinuxTStmpDataFor(year)

Cleaned data written to question1_data/2022_cleaned.csv
Cleaned data written to question1_data/2023_cleaned.csv
Cleaned data written to question1_data/2024_cleaned.csv
